In [9]:
import pandas as pd
import json
import os

In [10]:
def extract_text_from_sheet(sheet_df):
    text_list = []
    # Iterate through sheet_df.values.flatten()，
    # - .flatten() flattens all values in the DataFrame into a one-dimensional array.
    for value in sheet_df.values.flatten():
        # Check if the value is not NaN (missing value) & is of string type
        if pd.notna(value) and isinstance(value, str):
            
            # Filter data
            # Replace full-width brackets and other special characters
            value = value.replace('\uff08', '(').replace('\uff09', ')').replace('\uff1a', ':')
            # Remove all "AK" and "ES" strings to simplify reading
            value = value.replace("AK", "").replace("ES", "")
            
            # Split the value into lines and strip leading/trailing whitespace from each line
            # - .strip() removes leading and trailing whitespace from each line
            # - .split('\n') splits the string value into multiple substrings using the newline character '\n'
            text_list.extend([text.strip() for text in value.split('\n') if text.strip()])
            
    return text_list

In [11]:
# def test_process_excel_to_json():
#     input_file = 'test_input/Accessible King Room.xlsx'  # 替换为你的 Excel 文件名
#     output_folder = 'test_output'

#     # 调用函数处理 Excel 文件
#     json_file_path = process_excel_to_json(input_file, output_folder)
    
#     # 读取并打印生成的 JSON 文件内容
#     if json_file_path:
#         with open(json_file_path, 'r') as json_file:
#             json_content = json.load(json_file)
#             print(json.dumps(json_content, indent=4))
#     else:
#         print("No matching worksheets found.")
        
        
# test_process_excel_to_json()

In [12]:
def process_excel_to_json(file_path, output_folder):
    # 1.Load the Excel file
    # - ExcelFile(file_path) loads the Excel file from the given path and returns an ExcelFile object
    xl = pd.ExcelFile(file_path) 
    
    # 2. Extract the base name of the file from the given file path (without the file extension)
    # - os.path.basename(file_path) gets the base file name (with extension) from the file path
    # - os.path.splitext(os.path.basename(file_path)) splits the file name and extension
    # [0] gets the first element of the tuple, which is the file name (without extension)
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # 3. Create an output folder with the name of the Excel file and ensure it exists
    # - os.path.join(,) connects one or more path components, accepts one or more path strings (not limited to folder or file paths)
    specific_output_folder = os.path.join(output_folder, base_name)
    # - os.makedirs(,) creates the new output folder
    os.makedirs(specific_output_folder, exist_ok=True)
    
    # 4. Prepare a dictionary to hold extracted text
    all_text_data = {}
    
    # 5. Iterate over each worksheet and extract text
    # - xl.sheet_names is a list of all worksheet names
    for sheet_name in xl.sheet_names:
        # Only process worksheets with "Programming Details" in their name
        if "Programming Details" in sheet_name: 
            # - xl.parse(sheet_name) parses the worksheet into a DataFrame object
            df = xl.parse(sheet_name)
            all_text_data["programming details"] = extract_text_from_sheet(df)
    
    # 6. If there are no eligible worksheets, the function exits
    if not all_text_data:
        return None
    
    # 7. Convert the data to JSON format and save it
    json_output_path = os.path.join(specific_output_folder, 'input_data.json')
    # 7.1 Open the file
    # - with open(json_output_path, 'w') as json_file opens (or creates) the specified file in write mode
    with open(json_output_path, 'w') as json_file:
       # 7.2 Serialize and write the data to the file
        # - json.dump serializes a Python object to JSON format and writes it to a file
        json.dump(all_text_data, json_file, indent=4)
    
    return json_output_path

In [13]:

#! Process devices content and save in new format
def process_devices(split_data, output_folder):
    # Get devices content from the split_data dictionary, return an empty list if not found
    devices_content = split_data.get("devices", [])
    devices_data = []
    current_device = None
    # Iterate over the devices_content list
    for line in devices_content:
        # Skip lines containing '()', 'QTY:', or 'NAME:'
        if "(" in line and ")" in line:
            continue
        if line.startswith("QTY:"):
            continue 
        # if line == "NAME:":
        #     continue
        # Check if the line is a device shorthand name
        if line in ["KBSKTDIM", "KBSKTREL", "S2400IB2", "C300IBH", "H1RSMB", "H2RSMB", "H3RSMB", "H4RSMB", "H6RSMB", "6INPUT", "4OUTPUT"]:
            current_device = line  # Record the current device type
        elif current_device:
            # Add the current device type data to the list
            devices_data.append({
                "appearanceShortname": current_device,
                "deviceName": line
            })

    # Save devices_data as a JSON file
    devices_output_path = os.path.join(output_folder, "devices.json")
    with open(devices_output_path, 'w') as file:
        json.dump({"devices": devices_data}, file, indent=4)

# ! Process groups content and save in new format
def process_groups(split_data, output_folder):
    groups_content = split_data.get("groups", [])
    groups_data = []

    # 跳过包含 'TOTAL', 'SCENE', 'DEVICE CONTROL' 的行
    for line in groups_content:
        if line.startswith("TOTAL"):
            continue
        if "SCENE" in line:
            continue
        if line.startswith("DEVICE CONTROL"):
            continue
        if line.startswith("BLIND GROUP"):
            continue
        # 处理组名称
        if line and not line.startswith("BUTTON") and not any(keyword in line for keyword in ["ON", "OFF", "+"]):
            groups_data.append({
                "groupName": line,
                "devices": []
            })

    # 保存 groups_data 为 JSON 文件
    groups_output_path = os.path.join(output_folder, "groups.json")
    with open(groups_output_path, 'w') as file:
        json.dump({"groups": groups_data}, file, indent=4)

# ! Process remoteControls content and save in new format
def process_remote_controls(split_data, output_folder):
    # 获取 remoteControls 分类的内容
    remote_controls_content = split_data.get("remoteControls", [])

    # 初始化列表和变量
    remote_controls_data = []  # 存储处理后的远程控制数据
    current_remote = None  # 记录当前远程控制器的名称
    current_links = []  # 存储当前远程控制器的按钮链接
    remote_control_keywords = ["6IN", "4OUT", "S01", "S02", "S03", "S04", "S05", "S06","S07", "S09", "S10", "S11", "S12", "S13"]  # 远程控制关键字列表

    for line in remote_controls_content:
        # 跳过 TOTAL 行
        if line.startswith("TOTAL"):
            continue
        
        # 处理远程控制关键字
        if line.strip() in remote_control_keywords:
            # 如果 current_remote 不是 None，则将当前远程控制器数据添加到 remote_controls_data 列表中
            if current_remote:
                remote_controls_data.append({
                    "remoteName": current_remote,
                    "links": current_links
                })
            # 更新 current_remote 并去除多余空格
            current_remote = line.strip()
            # 清空 current_links 列表
            current_links = []
        
        # 处理 BUTTON 行
        elif line.startswith("BUTTON"):
            # 解析按钮索引和按钮名称
            parts = line.split(":")
            button_index = int(parts[0].replace("BUTTON", "").strip()) - 1
            button_name = parts[1].strip()

            # 根据按钮名称确定链接类型（场景、DND、组或设备）
            # 0 - 设备
            # 1 - 组
            # 2 - 场景
            # 3 - DND
            if "SCENE" in button_name:
                link_type = 2  # 场景类型
            elif "DND" in button_name:
                link_type = 3  # DND 类型
            elif "GROUP" in button_name:
                link_type = 1  # 组类型
            else:
                link_type = 0  # 默认为设备类型

            # 去除按钮名称中的前缀（例如，SCENE, DEVICE, GROUP）
            for prefix in ["SCENE ", "DEVICE ", "GROUP "]:
                if button_name.startswith(prefix):
                    button_name = button_name[len(prefix):].strip()

            # 将按钮的索引、类型和名称添加到 current_links 列表中
            current_links.append({
                "linkIndex": button_index,
                "linkType": link_type,
                "linkName": button_name
            })

    # 处理最后一个远程控制器
    if current_remote:
        remote_controls_data.append({
            "remoteName": current_remote,
            "links": current_links
        })

    # 保存 remote_controls_data 为 JSON 文件
    remote_controls_output_path = os.path.join(output_folder, "remoteControls.json")
    with open(remote_controls_output_path, 'w') as file:
        json.dump({"remoteControls": remote_controls_data}, file, indent=4)

#!   
def parse_scene_content(content_lines):
    contents = []
    for line in content_lines:
        parts = line.split()
        if len(parts) < 2:
            continue
        name = parts[0]
        status = parts[1]
        level = 100 if status == "ON" else 0
        if len(parts) > 2 and '+' in parts:
            try:
                level_index = parts.index('+') + 1
                level = int(parts[level_index].replace("%", ""))
            except (ValueError, IndexError):
                pass
        
        contents.append({
            "name": name,
            "status": status,
            "statusConditions": {
                "level": level
            }
        })
    return contents

def process_scenes(split_data, output_folder):
    scenes_content = split_data.get("scenes", [])
    scenes_data = []
    current_scene = None
    current_controls = []
    remote_control_keywords = [
    "BRIGHT",
    "OFF",
    "SOFT",
    "BATH ON",
    "BATH OFF",
    "BATH MOOD",
    "WELCOME",
    "OCCUPIED",
    "SMALL BATH ON",
    "SMALL BATH OFF",
    "SMALL BATH MOOD",
    "BED BRIGHT",
    "BED OFF",
    "BED SOFT",
    "DINING BRIGHT",
    "DINING OFF",
    "DINING SOFT",
    "LIVING BRIGHT",
    "LIVING OFF",
    "LIVING SOFT"
]

    for line in scenes_content:
        if line.startswith("TOTAL"):
            continue
        if line in remote_control_keywords:
            if current_scene:
                scenes_data.append({
                    "sceneName": current_scene,
                    "contents": parse_scene_content(current_controls)
                })
            current_scene = line
            current_controls = []
        else:
            current_controls.append(line)
    
    if current_scene:
        scenes_data.append({
            "sceneName": current_scene,
            "contents": parse_scene_content(current_controls)
        })

    scenes_output_path = os.path.join(output_folder, "scenes.json")
    with open(scenes_output_path, 'w') as file:
        json.dump({"scenes": scenes_data}, file, indent=4)
     
def summarize_all_files(output_folder):
    summary = {}
    for filename in ["devices.json", "groups.json", "scenes.json", "remoteControls.json"]:
        file_path = os.path.join(output_folder, filename)
        with open(file_path, 'r') as file:
            content = json.load(file)
        summary.update(content)
    
    # 保存 summary 为 JSON 文件并重命名为 result.json
    summary_output_path = os.path.join(output_folder, "result.json")
    with open(summary_output_path, 'w') as file:
        json.dump(summary, file, indent=4)
    
    # 删除不需要的 JSON 文件
    for filename in ["devices.json", "groups.json", "scenes.json", "remoteControls.json"]:
        file_path = os.path.join(output_folder, filename)
        if os.path.exists(file_path):
            os.remove(file_path)

In [14]:
def split_json_file(input_file_path, output_folder):
    with open(input_file_path, 'r') as file:
        data = json.load(file)
    content = data.get("programming details", [])
    split_keywords = {
        "devices": "KASTA DEVICE",
        "groups": "KASTA GROUP",
        "scenes": "KASTA SCENE",
        "remoteControls": "REMOTE CONTROL LINK"
    }
    split_data = {
        "devices": [],
        "groups": [],
        "scenes": [],
        "remoteControls": []
    }
    current_key = None
    for line in content:
        if line in split_keywords.values():
            current_key = next(key for key, value in split_keywords.items() if value == line)
            continue
        if current_key:
            split_data[current_key].append(line)
    os.makedirs(output_folder, exist_ok=True)
    for key, sub_content in split_data.items():
        output_file_path = os.path.join(output_folder, f"{key}.json")
        with open(output_file_path, 'w') as file:
            json.dump({key: sub_content}, file, indent=4)
            
    process_devices(split_data, output_folder)
    process_groups(split_data, output_folder)
    process_scenes(split_data, output_folder)
    process_remote_controls(split_data, output_folder)
    summarize_all_files(output_folder)

In [15]:
def batch_process_excel_to_json(input_folder, output_folder):
    # Iterate through each file in the input_folder
    for file_name in os.listdir(input_folder):
        # Check if the file ends with '.xlsx'
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(input_folder, file_name) # construct the full file path
            result = process_excel_to_json(file_path, output_folder)
            # If result is not None, JSON file was generated
            if result:
                print(f"JSON file has been generated: {result}")
                # dirname(result) - get the *directory part* of the specified path
                split_json_file(result, os.path.dirname(result))
            else:
                print(f"No matching worksheets found: {file_name}")

In [16]:
input_folder = 'input'
output_folder = 'output'

batch_process_excel_to_json(input_folder, output_folder)

JSON file has been generated: output/Standard King Room/input_data.json
JSON file has been generated: output/Executive Double Corner Room/input_data.json
JSON file has been generated: output/Executive Suite Room/input_data.json
JSON file has been generated: output/Executive Double Room/input_data.json
JSON file has been generated: output/Accessible King Room/input_data.json
JSON file has been generated: output/Standard Double Room/input_data.json
JSON file has been generated: output/Executive King Corner Room/input_data.json
JSON file has been generated: output/Executive King Room/input_data.json
